## Ensemble of models

1. deberta-large-text-scls-fgm-add2 Score: 0.604

2. deberta-v3-large-text-4cls-mlm-fgm01 Score: 0.602

3. robert-base / Score:0.649


#### If this notebook is helpful, please upvote the original versions:


## Content

> I tried to get the same result for each model, but by unifying the actions and removing the excess.
>  
> For these models, I use the same prepare_input and inference_fn.

```
def prepare_input(cfg, text, text_2=None):
    inputs = cfg.tokenizer(text, text_2,
                           padding="max_length",
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           truncation=True)

    [...]

def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
            
        with torch.no_grad():
            output = model(inputs)
        
    [...]
```


# 1. Import & Def & Set & Load

In [ ]:
import gc
import os
import pickle
import glob

from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs

import numpy as np
import pandas as pd

from tqdm import tqdm

import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Parameter
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer, AutoConfig

import warnings
warnings.simplefilter('ignore')

import re
from datasets import Dataset
import copy
import time
import random
import joblib

from torch.utils.data import  DataLoader
# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
def fetch_essay(essay_id: str, txt_dir: str):
    essay_path = os.path.join(COMP_DIR + txt_dir, essay_id + '.txt')
    essay_text = open(essay_path, 'r').read()
    
    return essay_text


def prepare_input(cfg, text, text_2=None):
    inputs = cfg.tokenizer(text, text_2,
                           padding="max_length",
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           truncation=True)

    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
        
    return inputs


def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
            
        with torch.no_grad():
            output = model(inputs)
        
        preds.append(F.softmax(output).to('cpu').numpy())

    return np.concatenate(preds)  


def show_gradient(df, n_row=None):
    if not n_row:
        n_row = 5

    return df.head(n_row) \
                .assign(all_mean=lambda x: x.mean(axis=1)) \
                    .style.background_gradient(cmap=cm, axis=1)

In [ ]:
def get_essay(essay_id):
    essay_path = os.path.join(TEST_DIR, f"{essay_id}.txt")
    essay_text = open(essay_path, 'r').read()
    return essay_text

def softmax(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div

def get_score(y_true, y_pred):
    y_pred = softmax(y_pred)
    score = log_loss(y_true, y_pred)
    return round(score, 5)

In [ ]:
pd.set_option('display.precision', 4)
cm = sns.light_palette('green', as_cmap=True)
props_param = "color:white; font-weight:bold; background-color:green;"

N_ROW = 10

COMP_DIR = "../input/feedback-prize-effectiveness/"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
TRAIN_DIR = "../input/feedback-prize-effectiveness/train"
TEST_DIR = "../input/feedback-prize-effectiveness/test"

In [ ]:
test_path = COMP_DIR + "test.csv"
train_path=COMP_DIR + "train.csv"
submission_path = COMP_DIR + "sample_submission.csv"

test_origin = pd.read_csv(test_path)
train_origin = pd.read_csv(train_path)
submission_origin = pd.read_csv(submission_path)

In [ ]:
test_origin = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
df=test_origin.copy()
df['essay_text'] = df['essay_id'].apply(get_essay)
df.head()

In [ ]:
from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text
df['discourse_text'] = df['discourse_text'].apply(lambda x : resolve_encodings_and_normalize(x))
df['essay_text'] = df['essay_text'].apply(lambda x : resolve_encodings_and_normalize(x))
#encoder = LabelEncoder()
#df['discourse_effectiveness'] = encoder.fit_transform(df['discourse_effectiveness'])

# with open("le.pkl", "wb") as fp:
#     joblib.dump(encoder, fp)

In [ ]:
df['text'] = df['essay_text']
disc_types = [
    "Claim",
    "Concluding Statement",
    "Counterclaim",
    "Evidence",
    "Lead",
    "Position",
    "Rebuttal",
]
cls_tokens_map = {label: f"[{label}]" for label in disc_types}
end_tokens_map = {label: f"[{label}]" for label in disc_types}
def find_positions(example):
    text = example["text"][0]

    # keeps track of what has already
    # been located
    min_idx = 0

    # stores start and end indexes of discourse_texts
    idxs = []

    for dt in example["discourse_text"]:
        # calling strip is essential
        matches = list(re.finditer(re.escape(dt.strip()), text))

        # If there are multiple matches, take the first one
        # that is past the previous discourse texts.
        if len(matches) > 1:
            for m in matches:
                if m.start() >= min_idx:
                    break
        # If no matches are found
        elif len(matches) == 0:
            idxs.append([-1])  # will filter out later
            continue
            # If one match is found
        else:
            m = matches[0]

        idxs.append([m.start(), m.end()])

        min_idx = m.start()

    return idxs


def tokenize(example):
    example["idxs"] = find_positions(example)

    text = example["text"][0]
    chunks = []
    labels = []
    prev = 0

    zipped = zip(
        example["idxs"],
        example["discourse_type"],
#         example["discourse_effectiveness"],
    )
    for idxs, disc_type in zipped:
        # when the discourse_text wasn't found
        if idxs == [-1]:
            continue

        s, e = idxs

        # if the start of the current discourse_text is not
        # at the end of the previous one.
        # (text in between discourse_texts)
        if s != prev:
            chunks.append(text[prev:s])
            prev = s

        # if the start of the current discourse_text is
        # the same as the end of the previous discourse_text
        if s == prev:
            chunks.append(cls_tokens_map[disc_type])
            chunks.append(text[s:e])
            chunks.append(end_tokens_map[disc_type])

        prev = e




    # at this point, labels is not the same shape as input_ids.
    # The following loop will add -100 so that the loss function
    # ignores all tokens except CLS tokens









    a=" ".join(chunks)
    example["text"][0]=a








    return example
# %%
grouped = df.groupby(["essay_id"]).agg(list)

num=range(0,grouped.shape[0])
grouped["num"]=num
ds = Dataset.from_pandas(grouped)
ds = ds.map(
    tokenize,
    batched=False,

)
x=ds["text"]

for i in range(df.shape[0]):
    a1=grouped.loc[df["essay_id"][i]]["num"]
    b1=x[a1][0]
    df["text"][i]=b1

del ds,grouped,x

In [ ]:
df["text"][1]

# 3. Extract predictions

## 3.1 DeBerta

In [ ]:
MODEL_DIR='../input/large-text-scls-fgmadd2'
MODEL_PATHS = [
    f'{MODEL_DIR}/Loss-Fold-0.bin',
    f'{MODEL_DIR}/Loss-Fold-1.bin',
    f'{MODEL_DIR}/Loss-Fold-2.bin',
    f'{MODEL_DIR}/Loss-Fold-3.bin',
    f'{MODEL_DIR}/Loss-Fold-4.bin',
]
TRAIN_DIR = "../input/feedback-prize-effectiveness/train"
TEST_DIR = "../input/feedback-prize-effectiveness/test"
CONFIG = dict(
    seed = 42,
    model_name = '../input/deberta-v3-large/deberta-v3-large',
    test_batch_size = 16,
    max_length = 512,
    num_classes = 3,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    gradient_checkpoint=False
)

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

In [ ]:
with open(f"../input/{MODEL_DIR}/le.pkl", "rb") as fp:
    encoder = joblib.load(fp)
    
encoder.classes_

In [ ]:
class FeedBackDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.discourse_type = df['discourse_type'].values
        self.discourse = df['discourse_text'].values
        self.essay = df['essay_text'].values
        self.essay_type = df['text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        discourse_type = self.discourse_type[index]
        discourse = self.discourse[index]
        essay_type = self.essay_type[index]
        essay = self.essay[index]
        text = discourse_type+' '+discourse +self.tokenizer.sep_token+essay_type
        inputs = self.tokenizer.encode_plus(
                        text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }

In [ ]:
class Collate:
    def __init__(self, tokenizer, isTrain=True):
        self.tokenizer = tokenizer
        self.isTrain = isTrain
        # self.args = args

    def __call__(self, batch):
        output = dict()
        output["input_ids"] = [sample["input_ids"] for sample in batch]
        output["attention_mask"] = [sample["attention_mask"] for sample in batch]
        if self.isTrain:
            output["target"] = [sample["target"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["input_ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["input_ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["input_ids"]]
            output["attention_mask"] = [s + (batch_max - len(s)) * [0] for s in output["attention_mask"]]
        else:
            output["input_ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["input_ids"]]
            output["attention_mask"] = [(batch_max - len(s)) * [0] + s for s in output["attention_mask"]]

        # convert to tensors
        output["input_ids"] = torch.tensor(output["input_ids"], dtype=torch.long)
        output["attention_mask"] = torch.tensor(output["attention_mask"], dtype=torch.long)
        if self.isTrain:
            output["target"] = torch.tensor(output["target"], dtype=torch.long)

        return output

collate_fn = Collate(CONFIG["tokenizer"], isTrain=False)

In [ ]:


test_dataset = FeedBackDataset(df, CONFIG['tokenizer'], max_length=CONFIG['max_length'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'],
                         num_workers=2, shuffle=False,collate_fn=collate_fn, pin_memory=True)

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings


class MeanMaxPooling(nn.Module):
    def __init__(self):
        super(MeanMaxPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        mean_pooling_embeddings = torch.mean(last_hidden_state, 1)
        _, max_pooling_embeddings = torch.max(last_hidden_state, 1)
        mean_max_embeddings = torch.cat((mean_pooling_embeddings, max_pooling_embeddings), 1)
        return mean_max_embeddings


class LSTMPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_lstm):
        super(LSTMPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_lstm = hiddendim_lstm
        self.lstm = nn.LSTM(self.hidden_size, self.hiddendim_lstm, batch_first=True)
        self.dropout = nn.Dropout(0.1)

    def forward(self, all_hidden_states):
        ## forward
        hidden_states = torch.stack([all_hidden_states[layer_i][:, 0].squeeze()
                                     for layer_i in range(1, self.num_hidden_layers + 1)], dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out, _ = self.lstm(hidden_states, None)
        out = self.dropout(out[:, -1, :])
        return out


class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights=None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
            torch.tensor([1] * (num_hidden_layers + 1 - layer_start), dtype=torch.float)
        )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor * all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average

In [ ]:
class FeedBackModel(nn.Module):
    def __init__(self, model_name):
        super(FeedBackModel, self).__init__()
        self.cfg = CONFIG
        self.config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name,config=self.config)
        # gradient checkpointing  梯度检查点
        if CONFIG['gradient_checkpoint']:
            self.model.gradient_checkpointing_enable()
            print(f"Gradient Checkpointing: {self.model.is_gradient_checkpointing}")
        self.bilstm = nn.LSTM(self.config.hidden_size, (self.config.hidden_size) // 2, num_layers=2,
                              dropout=self.config.hidden_dropout_prob, batch_first=True,
                              bidirectional=True)

        self.dropout = nn.Dropout(0.2)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        #-----------------
        #self.pooler=WeightedLayerPooling(self.config.num_hidden_layers)
        #self.pooler=MeanPooling()

        self.output = nn.Sequential(
            nn.Linear(self.config.hidden_size, CONFIG['num_classes'])
            # nn.Linear(256, self.cfg.target_size)
        )

    def loss(self, outputs, targets):
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs, targets)
        return loss

    def monitor_metrics(self, outputs, targets):
        device = targets.get_device()
        # print(outputs)
        # print(targets)
        mll = log_loss(
            targets.cpu().detach().numpy(),
            softmax(outputs.cpu().detach().numpy()),
            labels=[0, 1, 2],
        )
        return mll

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    # def forward(self, ids, mask):
    #     out = self.model(input_ids=ids, attention_mask=mask,
    #                      output_hidden_states=False)
    #     out = self.pooler(out.last_hidden_state, mask)
    #     out = self.drop(out)
    #     outputs = self.fc(out)
    #     return outputs
    def forward(self, ids, mask, token_type_ids=None, targets=None):
        if token_type_ids:
            transformer_out = self.model(ids, mask, token_type_ids)
        else:
            transformer_out = self.model(ids, mask)

        # LSTM/GRU header
        # all_hidden_states = torch.stack(transformer_out[1])
        # sequence_output = self.pooler(all_hidden_states)3
#         all_hidden_states = torch.stack(transformer_out[1])

#         concatenate_pooling = torch.cat(
#             (all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]), -1
#         )
#         sequence_output = concatenate_pooling[:, 0]
        # simple CLS
        #transformer_out = self.pooler(transformer_out.last_hidden_state,mask)
        #sequence_output = transformer_out
        sequence_output = transformer_out[0][:, 0, :]
        #all_hidden_states = torch.stack(transformer_out[1])
        #sequence_output=self.pooler(all_hidden_states)

        # Main task
        logits1 = self.output(self.dropout1(sequence_output))
        logits2 = self.output(self.dropout2(sequence_output))
        logits3 = self.output(self.dropout3(sequence_output))
        logits4 = self.output(self.dropout4(sequence_output))
        logits5 = self.output(self.dropout5(sequence_output))
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5

        if targets is not None:
            metric = self.monitor_metrics(logits, targets)
            return logits

        return logits

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        outputs = F.softmax(outputs, dim=1)
        PREDS.append(outputs.cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = FeedBackModel(CONFIG['model_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
        del model, preds
        torch.cuda.empty_cache()    
        gc.collect()
    
#     final_preds = np.array(final_preds)
#     final_preds = np.mean(final_preds, axis=0)


    return final_preds

In [ ]:
deberta_predictions = inference(MODEL_PATHS, test_loader, CONFIG['device'])

In [ ]:

deb_ineffective = []
deb_effective = []
deb_adequate = []

for x in deberta_predictions:
    deb_ineffective.append(x[:, 2])
    deb_adequate.append(x[:, 0])
    deb_effective.append(x[:, 1])

In [ ]:
# deberta_predictions = np.array(deberta_predictions)
# deberta_predictions = np.mean(deberta_predictions, axis=0)
# deberta_predictions.head()

In [ ]:
deberta_predictions


In [ ]:
deb_ineffective = pd.DataFrame(deb_ineffective).T

show_gradient(
    deb_ineffective,
    N_ROW)

In [ ]:
deb_adequate = pd.DataFrame(deb_adequate).T

show_gradient(
    deb_adequate,
    N_ROW)

In [ ]:
deb_effective = pd.DataFrame(deb_effective).T

show_gradient(
    deb_effective,
    N_ROW)

## 3.2 base

In [ ]:
MODEL_DIR='../input/large-text-4cls-mlm-fgm01'
MODEL_PATHS = [
    f'{MODEL_DIR}/Loss-Fold-0.bin',
    f'{MODEL_DIR}/Loss-Fold-1.bin',
    f'{MODEL_DIR}/Loss-Fold-2.bin',
    f'{MODEL_DIR}/Loss-Fold-3.bin',
#     f'{MODEL_DIR}/Loss-Fold-4.bin',
]
TRAIN_DIR = "../input/feedback-prize-effectiveness/train"
TEST_DIR = "../input/feedback-prize-effectiveness/test"
CONFIG = dict(
    seed = 42,
    model_name = '../input/fpdebertalarge',
    test_batch_size = 16,
    max_length = 512,
    num_classes = 3,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    gradient_checkpoint=False
)

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

In [ ]:
class FeedBackDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.discourse_type = df['discourse_type'].values
        self.discourse = df['discourse_text'].values
        self.essay = df['essay_text'].values
        self.essay_type = df['text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        discourse_type = self.discourse_type[index]
        discourse = self.discourse[index]
        essay_type = self.essay_type[index]
        essay = self.essay[index]
        text = discourse_type+' '+discourse +self.tokenizer.sep_token+essay_type
        inputs = self.tokenizer.encode_plus(
                        text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }

In [ ]:
class Collate:
    def __init__(self, tokenizer, isTrain=True):
        self.tokenizer = tokenizer
        self.isTrain = isTrain
        # self.args = args

    def __call__(self, batch):
        output = dict()
        output["input_ids"] = [sample["input_ids"] for sample in batch]
        output["attention_mask"] = [sample["attention_mask"] for sample in batch]
        if self.isTrain:
            output["target"] = [sample["target"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["input_ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["input_ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["input_ids"]]
            output["attention_mask"] = [s + (batch_max - len(s)) * [0] for s in output["attention_mask"]]
        else:
            output["input_ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["input_ids"]]
            output["attention_mask"] = [(batch_max - len(s)) * [0] + s for s in output["attention_mask"]]

        # convert to tensors
        output["input_ids"] = torch.tensor(output["input_ids"], dtype=torch.long)
        output["attention_mask"] = torch.tensor(output["attention_mask"], dtype=torch.long)
        if self.isTrain:
            output["target"] = torch.tensor(output["target"], dtype=torch.long)

        return output

collate_fn = Collate(CONFIG["tokenizer"], isTrain=False)

In [ ]:
test_dataset = FeedBackDataset(df, CONFIG['tokenizer'], max_length=CONFIG['max_length'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'],
                         num_workers=2, shuffle=False,collate_fn=collate_fn, pin_memory=True)

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings


class MeanMaxPooling(nn.Module):
    def __init__(self):
        super(MeanMaxPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        mean_pooling_embeddings = torch.mean(last_hidden_state, 1)
        _, max_pooling_embeddings = torch.max(last_hidden_state, 1)
        mean_max_embeddings = torch.cat((mean_pooling_embeddings, max_pooling_embeddings), 1)
        return mean_max_embeddings


class LSTMPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_lstm):
        super(LSTMPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_lstm = hiddendim_lstm
        self.lstm = nn.LSTM(self.hidden_size, self.hiddendim_lstm, batch_first=True)
        self.dropout = nn.Dropout(0.1)

    def forward(self, all_hidden_states):
        ## forward
        hidden_states = torch.stack([all_hidden_states[layer_i][:, 0].squeeze()
                                     for layer_i in range(1, self.num_hidden_layers + 1)], dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out, _ = self.lstm(hidden_states, None)
        out = self.dropout(out[:, -1, :])
        return out


class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights=None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
            torch.tensor([1] * (num_hidden_layers + 1 - layer_start), dtype=torch.float)
        )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor * all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average

In [ ]:
class FeedBackModel(nn.Module):
    def __init__(self, model_name):
        super(FeedBackModel, self).__init__()
        self.cfg = CONFIG
        self.config = AutoConfig.from_pretrained(model_name,output_hidden_states=True)
        self.model = AutoModel.from_pretrained(model_name,config=self.config)
        # gradient checkpointing  梯度检查点
        if CONFIG['gradient_checkpoint']:
            self.model.gradient_checkpointing_enable()
            print(f"Gradient Checkpointing: {self.model.is_gradient_checkpointing}")
        self.bilstm = nn.LSTM(self.config.hidden_size, (self.config.hidden_size) // 2, num_layers=2,
                              dropout=self.config.hidden_dropout_prob, batch_first=True,
                              bidirectional=True)

        self.dropout = nn.Dropout(0.2)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        #-----------------
        #self.pooler=WeightedLayerPooling(self.config.num_hidden_layers)
        #self.pooler=MeanPooling()

        self.output = nn.Sequential(
            nn.Linear(self.config.hidden_size*4, CONFIG['num_classes'])
            # nn.Linear(256, self.cfg.target_size)
        )

    def loss(self, outputs, targets):
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs, targets)
        return loss

    def monitor_metrics(self, outputs, targets):
        device = targets.get_device()
        # print(outputs)
        # print(targets)
        mll = log_loss(
            targets.cpu().detach().numpy(),
            softmax(outputs.cpu().detach().numpy()),
            labels=[0, 1, 2],
        )
        return mll

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    # def forward(self, ids, mask):
    #     out = self.model(input_ids=ids, attention_mask=mask,
    #                      output_hidden_states=False)
    #     out = self.pooler(out.last_hidden_state, mask)
    #     out = self.drop(out)
    #     outputs = self.fc(out)
    #     return outputs
    def forward(self, ids, mask, token_type_ids=None, targets=None):
        if token_type_ids:
            transformer_out = self.model(ids, mask, token_type_ids)
        else:
            transformer_out = self.model(ids, mask)

        # LSTM/GRU header
        # all_hidden_states = torch.stack(transformer_out[1])
        # sequence_output = self.pooler(all_hidden_states)3
        all_hidden_states = torch.stack(transformer_out[1])

        concatenate_pooling = torch.cat(
            (all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]), -1
        )
        sequence_output = concatenate_pooling[:, 0]
        # simple CLS
        #transformer_out = self.pooler(transformer_out.last_hidden_state,mask)
        #sequence_output = transformer_out
        #sequence_output = transformer_out[0][:, 0, :]
        #all_hidden_states = torch.stack(transformer_out[1])
        #sequence_output=self.pooler(all_hidden_states)

        # Main task
        logits1 = self.output(self.dropout1(sequence_output))
        logits2 = self.output(self.dropout2(sequence_output))
        logits3 = self.output(self.dropout3(sequence_output))
        logits4 = self.output(self.dropout4(sequence_output))
        logits5 = self.output(self.dropout5(sequence_output))
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5

        if targets is not None:
            metric = self.monitor_metrics(logits, targets)
            return logits

        return logits

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        outputs = F.softmax(outputs, dim=1)
        PREDS.append(outputs.cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = FeedBackModel(CONFIG['model_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
        del model, preds
        torch.cuda.empty_cache()    
        gc.collect()
    
#     final_preds = np.array(final_preds)
#     final_preds = np.mean(final_preds, axis=0)


    return final_preds

In [ ]:
base_predicts = inference(MODEL_PATHS, test_loader, CONFIG['device'])

In [ ]:
base_predicts

In [ ]:
base_ineffective = []
base_effective = []
base_adequate = []

for x in base_predicts:
    base_ineffective.append(x[:, 2])
    base_adequate.append(x[:, 0])
    base_effective.append(x[:, 1])

In [ ]:
base_ineffective = pd.DataFrame(base_ineffective).T

show_gradient(
    base_ineffective,
    N_ROW)

In [ ]:
base_adequate = pd.DataFrame(base_adequate).T

show_gradient(
    base_adequate,
    N_ROW)

In [ ]:
base_effective = pd.DataFrame(base_effective).T

show_gradient(
    base_effective,
    N_ROW)

## 3.3 roberta


In [ ]:
class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.discourse = df['discourse'].values
        self.essay = df['essay'].values
        
    def __len__(self):
        return len(self.discourse)
    
    def __getitem__(self, item):
        discourse = self.discourse[item]
        essay = self.essay[item]
        
        inputs = prepare_input(self.cfg, discourse, essay)
        
        return inputs
        
class FeedBackModel(nn.Module):
    def __init__(self, model_path):
        super(FeedBackModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_path)
        self.linear = nn.Linear(768, 3)

    def forward(self, inputs):
        last_hidden_states = self.model(**inputs)[0][:, 0, :]
        outputs = self.linear(last_hidden_states)
        
        return outputs
model_list = pickle.load(
    open("../input/feedback-roberta-ep1/roberta_modellist_ep2.pkl", "rb")
)

class CFG:
    path = "../input/roberta-base/"
    n_fold = 5
    batch = 16
    max_len = 512
    num_workers = 2
    
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path)
df = test_origin.copy()

txt_sep = " "
df['discourse'] = df['discourse_type'].str.lower().str.strip() + txt_sep \
                + df['discourse_text'].str.lower().str.strip()

df['essay'] = df['essay_id'].transform(fetch_essay, txt_dir='test').str.lower().str.strip()
df.head()


test_dataset = TestDataset(CFG, df)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch,
                         shuffle=False, num_workers=CFG.num_workers,
                         pin_memory=True, drop_last=False)
roberta_predicts = []
for i in range(CFG.n_fold):
    model = model_list[i]
    
    prediction = inference_fn(test_loader, model, DEVICE)
    roberta_predicts.append(prediction)
    
    del model, prediction
    torch.cuda.empty_cache()    
    gc.collect()
    
del model_list
gc.collect()

rob_ineffective = []
rob_effective = []
rob_adequate = []

for x in roberta_predicts:
    rob_ineffective.append(x[:, 0])
    rob_adequate.append(x[:, 1])
    rob_effective.append(x[:, 2])

# list -> dataframe
rob_ineffective = pd.DataFrame(rob_ineffective).T
rob_adequate = pd.DataFrame(rob_adequate).T
rob_effective = pd.DataFrame(rob_effective).T

In [ ]:
roberta_predicts

# 4. Create submission

In [ ]:
level_names = ['deberta', 'base','rob']

ineffective_ = pd.concat(
    [deb_ineffective, base_ineffective,rob_ineffective],
    keys=level_names, axis=1
)

adequate_ = pd.concat(
    [deb_adequate, base_adequate,rob_adequate],
    keys=level_names, axis=1
)

effective_ = pd.concat(
    [deb_effective, base_effective,rob_effective],
    keys=level_names, axis=1
)

In [ ]:
show_gradient(
    ineffective_,
    N_ROW
)

In [ ]:
show_gradient(
    adequate_,
    N_ROW
)

In [ ]:
show_gradient(
    effective_,
    N_ROW
)

In [ ]:
submission = submission_origin.copy()

w_ = [.30,.60,.10]  
d_ = [('Ineffective', ineffective_),
      ('Adequate', adequate_),
      ('Effective', effective_)]

for x in d_:
    col_name, df = x
    submission[col_name] = pd.DataFrame(
        {col: df[col].mean(axis=1) for col in level_names}
    ).mul(w_).sum(axis=1)    

submission.head(N_ROW)

In [ ]:
# 0	a261b6e14276	0.0102	0.3887	0.5911
# 1	5a88900e7dc1	0.0309	0.8405	0.1185
# 2	9790d835736b	0.0217	0.6997	0.2686
# 3	75ce6d68b67b	0.0512	0.6365	0.3023
# 4	93578d946723	0.0399	0.6053	0.3448
# 5	2e214524dbe3	0.0099	0.3721	0.6080
# 6	84812fc2ab9f	0.0084	0.2796	0.7020
# 7	c668ff840720	0.0171	0.5888	0.3841
# 8	739a6d00f44a	0.0178	0.4030	0.5692
# 9	bcfae2c9a244	0.0132	0.6373	0.3395

In [ ]:
submission.to_csv('submission.csv',index=False)